In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
#from tensorflow.keras.utils import to_categorical ## specific  one I wanna use
from tensorflow.keras import utils
#from tensorflow.keras.layers import Input, Dense, Dropout  # Specific ones I wanna use, can just import whole module
from tensorflow.keras import layers 
from tensorflow.keras.models import Model
import h5py
import tables
import matplotlib.pyplot as plt
#import tensorflow as tf
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve
from sklearn.utils.class_weight import compute_class_weight


In [ ]:
x=1
y=3
x+y

In [2]:
# Import data from all 4 collision types.

input_file_1 = "/scratch/ys20884/ml_vars/TTToHadronic/df_ml_inputs.hd5"

with h5py.File(input_file_1, "r") as f:
    #print("Contents of the file:")
    for key in f.keys():
        print(key)

df1= pd.read_hdf(input_file_1, key="df")


input_file2 = "/scratch/ys20884/ml_vars/TTToSemiLeptonic/df_ml_inputs.hd5"

with h5py.File(input_file2, "r") as f:
    #print("Contents of the file:")
    for key in f.keys():
        print(key)

df2 = pd.read_hdf(input_file2, key="df")
print("Dataframe columns:")
print(df2.columns)

input_file3 = "/scratch/ys20884/ml_vars/TTTo2L2Nu/df_ml_inputs.hd5"

with h5py.File(input_file3, "r") as f:
    #print("Contents of the file:")
    for key in f.keys():
        print(key)

df3 = pd.read_hdf(input_file3, key="df")

input_file4 = "/scratch/ys20884/ml_vars/ttH125/df_ml_inputs.hd5"

with h5py.File(input_file4, "r") as f:
    #print("Contents of the file:")
    for key in f.keys():
        print(key)

df4 = pd.read_hdf(input_file4, key="df")

print(len(df1))
print(len(df2))
print(len(df3))
print(len(df4))

df
df
Dataframe columns:
Index(['dataset', 'entry', 'BiasedDPhi', 'DiJet_mass', 'HT',
       'InputMet_InputJet_mindPhi', 'InputMet_phi', 'InputMet_pt', 'MHT_phi',
       'MHT_pt', 'MinChi', 'MinOmegaHat', 'MinOmegaTilde',
       'boostedObject_area', 'boostedObject_eta', 'boostedObject_mass',
       'boostedObject_phi', 'boostedObject_pt', 'boostedTop_area',
       'boostedTop_eta', 'boostedTop_mass', 'boostedTop_phi', 'boostedTop_pt',
       'boostedTop_tag', 'boostedV_area', 'boostedV_eta', 'boostedV_mass',
       'boostedV_phi', 'boostedV_pt', 'boostedV_tag', 'cleanedBJet_eta',
       'cleanedBJet_phi', 'cleanedBJet_pt', 'cleanedJet_area',
       'cleanedJet_btagDeepB', 'cleanedJet_chHEF', 'cleanedJet_eta',
       'cleanedJet_mass', 'cleanedJet_neHEF', 'cleanedJet_phi',
       'cleanedJet_pt', 'hashed_filename', 'ncleanedBJet', 'ncleanedJet',
       'weight_nominal', 'xs_weight'],
      dtype='object')
df
df
12185
651387
39666
177858


In [3]:
# make one big dataframe, full data set long to practice with
df_total = df2.append(df1)
df_total = df_total.append(df3)
df_total = df_total.append(df4)
print(len(df_total))   

# Background is 0
# Signal is 1
df_process = df_total['dataset']
df_total['dataset'] = df_total['dataset'].replace({'ttH125': 1, 'TTToSemiLeptonic': 0, 'TTToHadronic': 0, 'TTTo2L2Nu':0})
df_total.head()
# Have done test cases, is working fine.
print(len(df_total))   


881096
881096


In [4]:
# drop all labels and the phi's
df_total.drop(['entry', 'BiasedDPhi', 'hashed_filename', 'weight_nominal', 'xs_weight', 
              'InputMet_phi', 'MHT_phi', 'boostedObject_phi', 'boostedTop_phi', 'boostedV_phi',
              'cleanedBJet_phi','cleanedJet_phi'], axis=1, inplace=True)#
df_total.columns            # Said BiasedDPhi looks wrong



Index(['dataset', 'DiJet_mass', 'HT', 'InputMet_InputJet_mindPhi',
       'InputMet_pt', 'MHT_pt', 'MinChi', 'MinOmegaHat', 'MinOmegaTilde',
       'boostedObject_area', 'boostedObject_eta', 'boostedObject_mass',
       'boostedObject_pt', 'boostedTop_area', 'boostedTop_eta',
       'boostedTop_mass', 'boostedTop_pt', 'boostedTop_tag', 'boostedV_area',
       'boostedV_eta', 'boostedV_mass', 'boostedV_pt', 'boostedV_tag',
       'cleanedBJet_eta', 'cleanedBJet_pt', 'cleanedJet_area',
       'cleanedJet_btagDeepB', 'cleanedJet_chHEF', 'cleanedJet_eta',
       'cleanedJet_mass', 'cleanedJet_neHEF', 'cleanedJet_pt', 'ncleanedBJet',
       'ncleanedJet'],
      dtype='object')

In [5]:
lst = []
for column in df_total:
    if df_total[column].dtype == 'object':   
        print(column, 'is a jet based variable')
        lst.append(column)
    elif column == 'dataset':    # dont want to drop dataset column (label)
        print('Need to keep labels')
        lst.append(column)
    else:
        df_total.drop([column], axis=1, inplace=True)
print(len(lst)-1, 'variables are left')    # -1 as - label
df_total.columns 

Need to keep labels
boostedObject_area is a jet based variable
boostedObject_eta is a jet based variable
boostedObject_mass is a jet based variable
boostedObject_pt is a jet based variable
boostedTop_area is a jet based variable
boostedTop_eta is a jet based variable
boostedTop_mass is a jet based variable
boostedTop_pt is a jet based variable
boostedTop_tag is a jet based variable
boostedV_area is a jet based variable
boostedV_eta is a jet based variable
boostedV_mass is a jet based variable
boostedV_pt is a jet based variable
boostedV_tag is a jet based variable
cleanedBJet_eta is a jet based variable
cleanedBJet_pt is a jet based variable
cleanedJet_area is a jet based variable
cleanedJet_btagDeepB is a jet based variable
cleanedJet_chHEF is a jet based variable
cleanedJet_eta is a jet based variable
cleanedJet_mass is a jet based variable
cleanedJet_neHEF is a jet based variable
cleanedJet_pt is a jet based variable
23 variables are left


Index(['dataset', 'boostedObject_area', 'boostedObject_eta',
       'boostedObject_mass', 'boostedObject_pt', 'boostedTop_area',
       'boostedTop_eta', 'boostedTop_mass', 'boostedTop_pt', 'boostedTop_tag',
       'boostedV_area', 'boostedV_eta', 'boostedV_mass', 'boostedV_pt',
       'boostedV_tag', 'cleanedBJet_eta', 'cleanedBJet_pt', 'cleanedJet_area',
       'cleanedJet_btagDeepB', 'cleanedJet_chHEF', 'cleanedJet_eta',
       'cleanedJet_mass', 'cleanedJet_neHEF', 'cleanedJet_pt'],
      dtype='object')

In [ ]:
# Need to find the longest array in any column for my first method

# for column in df_total:
#     for i in column:
#         df_total[column]
#    x_arr = np.max(df_total[column])

In [11]:
# loop through each column and find the maximum array length
max_lengths = {}
min_lengths = {}
for column in df_total:
    if df_total[column].dtype == 'object': 
        #print(column)
        max_lengths[column] = max([len(arr) for arr in df_total[column]])
        min_lengths[column] = min([len(arr) for arr in df_total[column]])
    else:
        pass

print('max lengths=', max_lengths)
print('min lenghts =', min_lengths)
arr_lengths = max(max_lengths.values())

max lengths= {'boostedObject_area': 2, 'boostedObject_eta': 2, 'boostedObject_mass': 2, 'boostedObject_pt': 2, 'boostedTop_area': 2, 'boostedTop_eta': 2, 'boostedTop_mass': 2, 'boostedTop_pt': 2, 'boostedTop_tag': 2, 'boostedV_area': 2, 'boostedV_eta': 2, 'boostedV_mass': 2, 'boostedV_pt': 2, 'boostedV_tag': 2, 'cleanedBJet_eta': 2, 'cleanedBJet_pt': 2, 'cleanedJet_area': 2, 'cleanedJet_btagDeepB': 2, 'cleanedJet_chHEF': 2, 'cleanedJet_eta': 2, 'cleanedJet_mass': 2, 'cleanedJet_neHEF': 2, 'cleanedJet_pt': 2}
min lenghts = {'boostedObject_area': 2, 'boostedObject_eta': 2, 'boostedObject_mass': 2, 'boostedObject_pt': 2, 'boostedTop_area': 2, 'boostedTop_eta': 2, 'boostedTop_mass': 2, 'boostedTop_pt': 2, 'boostedTop_tag': 2, 'boostedV_area': 2, 'boostedV_eta': 2, 'boostedV_mass': 2, 'boostedV_pt': 2, 'boostedV_tag': 2, 'cleanedBJet_eta': 2, 'cleanedBJet_pt': 2, 'cleanedJet_area': 2, 'cleanedJet_btagDeepB': 2, 'cleanedJet_chHEF': 2, 'cleanedJet_eta': 2, 'cleanedJet_mass': 2, 'cleanedJet_ne

In [7]:
# Define a function that selects the two largest elements of an array and pads it if needed
def modify_array(arr):
    if len(arr) == 0:
        return np.array([0,0])
    elif len(arr) == 1:
        return np.array([arr[0], 0])
    else:
        arr_sorted = np.sort(arr)
        return np.array([arr_sorted[-1], arr_sorted[-2]])

df_total.drop(['dataset'], axis=1, inplace=True) # so only padded data, no labels  

# This should have manipulated all data so is now fit for a ML model
for column in df_total:    
    if df_total[column].dtype == 'object':
        df_total[column] = df_total[column].apply(lambda x: modify_array(x))

        

In [ ]:
# loop through each column and find the maximum array length
max_lengths = {}
min_lengths = {}
for column in df_total:
    if df_total[column].dtype == 'object': 
        #print(column)
        max_lengths[column] = max([len(arr) for arr in df_total[column]])
        min_lengths[column] = min([len(arr) for arr in df_total[column]])
    else:
        pass

print('max lengths=', max_lengths)
print('min lenghts =', min_lengths)
arr_lengths = max(max_lengths.values())

In [8]:
df_total

,boostedObject_area,boostedObject_eta,boostedObject_mass,boostedObject_pt,boostedTop_area,boostedTop_eta,boostedTop_mass,boostedTop_pt,boostedTop_tag,boostedV_area,...,boostedV_tag,cleanedBJet_eta,cleanedBJet_pt,cleanedJet_area,cleanedJet_btagDeepB,cleanedJet_chHEF,cleanedJet_eta,cleanedJet_mass,cleanedJet_neHEF,cleanedJet_pt
0,"[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]",...,"[0, 0]","[0, 0]","[0, 0]","[0.48876953, 0.45874023]","[0.33007812, 0.03375244]","[0.7734375, 0.12402344]","[2.1699219, 1.4013672]","[26.78125, 11.1171875]","[0.05517578, 0.0]","[319.85184, 89.55973]"
1,"[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]",...,"[0, 0]","[-0.95703125, 0.0]","[141.625, 0.0]","[0.5185547, 0.5185547]","[0.8305664, 0.10760498]","[0.6875, 0.65625]","[2.5351562, 0.48168945]","[24.203125, 17.90625]","[0.171875, 0.12890625]","[158.16063, 141.62325]"
2,"[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]",...,"[0, 0]","[-0.3046875, 0.0]","[54.8125, 0.0]","[0.5185547, 0.47875977]","[0.84277344, 0.026428223]","[0.8359375, 0.625]","[0.5942383, -0.3046875]","[53.90625, 6.03125]","[0.0859375, 0.060546875]","[306.7605, 54.809956]"
3,"[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]",...,"[0, 0]","[0, 0]","[0, 0]","[0.4987793, 0.48876953]","[0.15124512, 0.04321289]","[0.6640625, 0.609375]","[1.5175781, 1.0400391]","[31.234375, 16.203125]","[0.1015625, 0.049316406]","[186.68669, 108.03608]"
4,"[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]",...,"[0, 0]","[0, 0]","[0, 0]","[0.50878906, 0.4987793]","[0.1282959, 0.022079468]","[0.859375, 0.7109375]","[1.2236328, 1.03125]","[16.625, 9.2578125]","[0.15625, 0.064453125]","[117.50915, 96.34649]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177853,"[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]",...,"[0, 0]","[0, 0]","[0, 0]","[0.5288086, 0.5288086]","[0.17285156, 0.114868164]","[0.8984375, 0.53125]","[1.6572266, 1.2070312]","[18.046875, 9.8125]","[0.11230469, 0.0027770996]","[317.70575, 103.69638]"
177854,"[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]",...,"[0, 0]","[0, 0]","[0, 0]","[0.5683594, 0.5385742]","[0.3095703, 0.20300293]","[0.8515625, 0.7734375]","[1.5878906, 1.2285156]","[24.90625, 18.234375]","[0.2734375, 0.23632812]","[153.2326, 134.28952]"
177855,"[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]",...,"[0, 0]","[1.2978515625, 0.0]","[133.75, 0.0]","[0.50878906, 0.4987793]","[0.9790039, 0.21130371]","[0.8203125, 0.7734375]","[1.8935547, 1.6523438]","[23.359375, 17.625]","[0.5078125, 0.4296875]","[225.27489, 195.30655]"
177856,"[2.00390625, 0.0]","[-1.870849609375, 0.0]","[87.1875, 0.0]","[263.5, 0.0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[2.00390625, 0.0]",...,"[0.9345703125, 0.0]","[-1.8583984375, 0.0]","[73.1875, 0.0]","[0.5385742, 0.5385742]","[0.70458984, 0.13659668]","[0.7578125, 0.7265625]","[-0.9692383, -1.6191406]","[18.5, 18.3125]","[0.4609375, 0.28515625]","[160.7482, 154.11227]"


In [ ]:
# 16 pad

# i=0
# for column in df_total: 
#     if df_total[column].dtype == 'object': 
#         i+=1
#         print(i)
#         df_total[column] = df_total[column].apply(lambda x: np.pad(x, (0, arr_lengths-len(x)), mode='constant'))
#     else:
#         pass

# df_total.drop(['dataset'], axis=1, inplace=True) # so only padded data, no labels    
#df_total['boostedObject_area'] = df_total['boostedObject_area'].apply(lambda x: np.pad(x, (0, arr_lengths-len(x)), mode='constant'))

In [12]:
# Reshape into 3D array, instead of 2D df
leading_2_arr_3d = np.array([row.tolist() for row in df_total.values]).reshape(len(df_total), arr_lengths, df_total.shape[1])
# only takes like 17 seconds

In [13]:
# Save to HDF5 file
with h5py.File('leading_2_arr_3d.h5', 'w') as f:
    f.create_dataset('array', data=leading_2_arr_3d)


In [ ]:
df_total.shape

In [ ]:
# df_labels = df2['dataset'].append(df1['dataset'])
# df_labels = df_labels.append(df3['dataset'])
# df_labels = df_labels.append(df4['dataset'])

# df_labels = df_labels.replace({'ttH125': 1, 'TTToSemiLeptonic': 0, 'TTToHadronic': 0, 'TTTo2L2Nu':0})


# # Save to HDF5 file
# with h5py.File('4_event_types_labels.h5', 'w') as f:
#     f.create_dataset('array', data=df_labels)

# This became really complicated for some reason, and isn't more efficient to a point worth working out

In [ ]:
# Load padded data
#with h5py.File('padded_arr_3d.h5', 'r') as f:
 #   padded_arr_3d = f['array'][:]
#padded_arr_3d

In [ ]:
# df_weights = pd.concat([df1['xs_weight'], df2['xs_weight'], df3['xs_weight'], df4['xs_weight']], axis=1)
# df_weights.columns = ['df1_xs_weight', 'df2_xs_weight', 'df3_xs_weight', 'df4_xs_weight']
# (df_weights)

# with h5py.File('4_event_types_weights.h5', 'w') as f:
#     f.create_dataset('array', data=df_weights)


In [ ]:
#model.save('trained_model.h5')

#from tensorflow.keras.models import load_model
#loaded_model = load_model('trained_model.h5')